In [ ]:
!pip install gradio requests openai


In [ ]:
from __future__ import annotations
import os
import re
import json
import shlex
import textwrap
from typing import Optional, Dict, Any, Tuple

try:
    import openai
except Exception:
    openai = None

import requests
import gradio as gr

OPENAI_API_KEY = os.environ.get("OPEN AI API KEY")
OPENAI_MODEL = os.environ.get("OPENAI_MODEL", "gpt-4o-mini")
if OPENAI_API_KEY and openai is not None:
    try:
        openai.api_key = OPENAI_API_KEY
    except Exception:
        pass

def mask_token(token: str) -> str:
    if not token:
        return token
    if len(token) <= 8:
        return "*" * len(token)
    return token[:4] + "..." + token[-4:]

def build_parser_prompt(instruction: str) -> str:
    examples = [
        {
            "instr": "Test a GET request to https://api.example.com/users with token abc123",
            "json": {
                "method": "GET",
                "url": "https://api.example.com/users",
                "headers": {"Authorization": "Bearer abc123"},
                "auth": {"type": "bearer", "token": "abc123"},
                "body": None,
            },
        },
        {
            "instr": "Send a POST to /login with JSON {\"user\":\"alice\",\"pass\":\"mypassword\"}",
            "json": {
                "method": "POST",
                "url": "/login",
                "headers": {"Content-Type": "application/json"},
                "auth": None,
                "body": {"user": "alice", "pass": "mypassword"},
            },
        },
    ]

    prompt = textwrap.dedent(
        """
You are a strict parser that converts a developer's natural-language instruction into a JSON object only. Output JSON with exact keys: method, url, headers, auth, body. Do not output explanatory text.
"""
    )

    for ex in examples:
        prompt += "INSTRUCTION: " + ex["instr"] + "\n"
        prompt += "JSON: " + json.dumps(ex["json"]) + "\n\n"

    prompt += "INSTRUCTION: " + instruction + "\n"
    prompt += "JSON: "
    return prompt

def _extract_message_text_from_openai_response(resp: Any) -> Optional[str]:
    try:
        if isinstance(resp, dict):
            choices = resp.get("choices") or []
        else:
            choices = getattr(resp, "choices", [])
        if not choices:
            return None
        first = choices[0]
        if isinstance(first, dict):
            msg = first.get("message")
            if isinstance(msg, dict):
                return msg.get("content")
            return first.get("text")
        else:
            if hasattr(first, "message"):
                return getattr(first.message, "content", None)
            return getattr(first, "text", None)
    except Exception:
        return None

def extract_json_from_text(text: str) -> Optional[Dict[str, Any]]:
    if not isinstance(text, str):
        return None
    s = text.strip()
    start = s.find('{')
    if start == -1:
        try:
            return json.loads(s)
        except Exception:
            return None
    stack = []
    for i in range(start, len(s)):
        c = s[i]
        if c == '{':
            stack.append(i)
        elif c == '}':
            stack.pop()
            if not stack:
                candidate = s[start:i+1]
                try:
                    return json.loads(candidate)
                except Exception:
                    break
    try:
        return json.loads(s)
    except Exception:
        return None

def call_llm_parser(instruction: str, model: str = OPENAI_MODEL) -> Tuple[Optional[Dict[str, Any]], Optional[str]]:
    if openai is None or OPENAI_API_KEY is None:
        return None, "OpenAI not configured."
    prompt = build_parser_prompt(instruction)
    try:
        if hasattr(openai, "ChatCompletion"):
            resp = openai.ChatCompletion.create(model=model, messages=[{"role": "system", "content": "You are a parser that outputs only JSON."}, {"role": "user", "content": prompt}], temperature=0.0, max_tokens=500)
        elif hasattr(openai, "Completion"):
            resp = openai.Completion.create(model=model, prompt=prompt, temperature=0.0, max_tokens=500)
        else:
            return None, "No supported OpenAI client interface found."
        text = _extract_message_text_from_openai_response(resp)
        if not text:
            return None, "LLM returned empty response"
        parsed = extract_json_from_text(text)
        if parsed is None:
            return None, f"LLM returned unparsable JSON: {text[:200]}"
        return parsed, text
    except Exception as e:
        return None, str(e)

def fallback_parse(instruction: str) -> Dict[str, Any]:
    inst = instruction.strip()
    method = "GET"
    m = re.search(r"\b(GET|POST|PUT|DELETE|PATCH|OPTIONS|HEAD)\b", inst, re.IGNORECASE)
    if m:
        method = m.group(1).upper()
    url = ""
    u = re.search(r"https?://[\w\-\._~:/?#\[\]@!$&'()*+,;=%]+", inst)
    if u:
        url = u.group(0)
    else:
        p = re.search(r"(/[-\w@:\.%_\+~#=\/{}]+)", inst)
        if p:
            url = p.group(1)
    headers = {}
    auth = None
    tok = None
    m_tok = re.search(r"token\s*[:=]?\s*([A-Za-z0-9\-\._~\+\/=]+)", inst, re.IGNORECASE)
    if m_tok:
        tok = m_tok.group(1)
        auth = {"type": "bearer", "token": tok}
        headers["Authorization"] = f"Bearer {tok}"
    else:
        m_apikey = re.search(r"api[-_ ]?key\s*[:=]?\s*([A-Za-z0-9\-\._~\+\/=]+)", inst, re.IGNORECASE)
        if m_apikey:
            tok = m_apikey.group(1)
            auth = {"type": "api_key", "token": tok}
            headers["Authorization"] = f"ApiKey {tok}"
    body = None
    json_region = None
    j = re.search(r"(\{[\s\S]*\})", inst)
    if j:
        json_region = j.group(1)
        try:
            body = json.loads(json_region)
            headers.setdefault("Content-Type", "application/json")
        except Exception:
            body = json_region
            headers.setdefault("Content-Type", "text/plain")
    header_pattern = re.compile(r"([A-Za-z-]+)\s*:\s*([^,;\n]+)")
    for header_match in header_pattern.finditer(inst):
        k = header_match.group(1).strip()
        v = header_match.group(2).strip()
        if json_region and header_match.start() >= inst.find(json_region) and header_match.end() <= inst.find(json_region) + len(json_region):
            continue
        if k.lower() == "authorization":
            headers["Authorization"] = v
        else:
            headers[k] = v
    return {"method": method, "url": url, "headers": headers, "auth": auth, "body": body}

def build_curl(parsed: Dict[str, Any]) -> str:
    method = parsed.get("method", "GET").upper()
    url = parsed.get("url", "")
    headers = parsed.get("headers") or {}
    body = parsed.get("body")
    parts = ["curl"]
    parts.extend(["-X", shlex.quote(method)])
    for k, v in headers.items():
        parts.extend(["-H", shlex.quote(f"{k}: {v}")])
    if body is not None:
        if isinstance(body, (dict, list)):
            body_str = json.dumps(body)
        else:
            body_str = str(body)
        parts.extend(["-d", shlex.quote(body_str)])
    parts.append(shlex.quote(url))
    return " \\\n".join(parts)

def run_request(parsed: Dict[str, Any], timeout: int = 10) -> Tuple[int, Dict[str, Any], str]:
    url = parsed.get("url")
    method = parsed.get("method", "GET").upper()
    headers = parsed.get("headers") or {}
    body = parsed.get("body")
    if not url:
        return 0, {}, "Missing URL"
    if url.startswith("/"):
        return 0, {}, "URL appears to be a path only. Provide a full URL including scheme and host."
    try:
        if body is None:
            resp = requests.request(method, url, headers=headers, timeout=timeout)
        else:
            if isinstance(body, (dict, list)):
                resp = requests.request(method, url, headers=headers, json=body, timeout=timeout)
            else:
                resp = requests.request(method, url, headers=headers, data=str(body), timeout=timeout)
        result_headers = dict(resp.headers)
        try:
            result_body = resp.json()
            body_text = json.dumps(result_body, indent=2)
        except Exception:
            body_text = resp.text
        return resp.status_code, result_headers, body_text
    except Exception as e:
        return 0, {}, str(e)

def generate_request_from_instruction(instruction: str) -> Tuple[Dict[str, Any], str, str]:
    if not instruction or not instruction.strip():
        return {}, "", "Please provide an instruction."
    if OPENAI_API_KEY and openai is not None:
        parsed, raw = call_llm_parser(instruction)
        if parsed:
            curl = build_curl(parsed)
            parsed_display = json.loads(json.dumps(parsed))
            if parsed_display.get("auth") and parsed_display["auth"].get("token"):
                parsed_display["auth"]["token"] = mask_token(parsed_display["auth"]["token"])
            return parsed_display, curl, "Parsed with LLM"
    parsed = fallback_parse(instruction)
    curl = build_curl(parsed)
    parsed_display = json.loads(json.dumps(parsed))
    if parsed_display.get("auth") and parsed_display["auth"].get("token"):
        parsed_display["auth"]["token"] = mask_token(parsed_display["auth"]["token"])
    return parsed_display, curl, "Parsed with fallback rule-based parser"

EXAMPLES = [
    "Test a GET request to https://api.example.com/users with token abc123",
    "Send a POST to /login with JSON {\"user\":\"alice\",\"pass\":\"mypassword\"}",
    "Test GET /profile with API key=XYZ and header Accept: application/xml",
]

def build_ui() -> gr.Blocks:
    with gr.Blocks(title="API Test — Request Setup Automation") as demo:
        gr.Markdown("# API Test — Request Setup Automation")
        gr.Markdown("Describe the API request you want and the assistant will generate a structured request and a cURL command.")
        with gr.Row():
            inp = gr.Textbox(label="Instruction", placeholder='e.g. Send a POST to /login with JSON {"user":"alice"}', lines=3)
            with gr.Column(scale=1):
                gen_btn = gr.Button("Generate Request")
                run_btn = gr.Button("Run Request")
                export_dropdown = gr.Dropdown(choices=["cURL", "Postman (raw JSON)"], value="cURL", label="Export as")
        with gr.Row():
            parsed_out = gr.JSON(label="Structured Request")
            curl_out = gr.Textbox(label="cURL / Export", lines=5)
        status = gr.Textbox(label="Status", interactive=False)
        response_box = gr.Textbox(label="Live Response (if run)", lines=10, interactive=False)
        gr.Examples(examples=EXAMPLES, inputs=inp)
        def on_generate(instruction: str):
            parsed, curl, status_msg = generate_request_from_instruction(instruction)
            return parsed, curl, status_msg
        gen_btn.click(on_generate, inputs=[inp], outputs=[parsed_out, curl_out, status])
        def on_run(parsed_json):
            if not parsed_json:
                return "", "No parsed request to run. Generate one first."
            try:
                status_code, resp_headers, body_text = run_request(parsed_json)
                if status_code == 0:
                    return "", f"Request failed: {body_text}"
                resp_display = f"Status: {status_code}\nHeaders: {json.dumps(resp_headers, indent=2)}\nBody:\n{body_text}"
                return "Ran request successfully.", resp_display
            except Exception as e:
                return "", f"Error running request: {str(e)}"
        run_btn.click(on_run, inputs=[parsed_out], outputs=[status, response_box])
    return demo

if __name__ == "__main__":
    demo = build_ui()
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a92e96b5beca78330b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
